In [1]:
import pandas as pd 
import numpy as np 
from statsmodels.formula.api import ols 
from statsmodels.formula.api import logit 

In [24]:
general = pd.read_csv('C:/Users/박예슬/OneDrive/데이터공부/project/Fund Raising 성공의 조건/final_data/general_final.csv', encoding = 'utf-8')

In [25]:
general.head()

,chid,url,scrape_date,title,created_date,funds_raised,funds_target,picture,org_picture,counts_status_code,...,recent_time_window,funds_raised_dollar,funds_target_dollar,amount_raised_unattributed_dollar,char_age,success,dum_success,group_category,total_comments,update_counts
0,1,https://www.gofundme.com/f/-chris-beaty-memori...,2020-11-03,Chris Beaty Memorial Fund,2020-06-03,175083.0,200000.0,True,False,200,...,48 hours,175083,200000.0,0.0,153,False,0,Funerals & Memorials,74.0,1.0
1,2,https://www.gofundme.com/f/1-prosent-chance,2020-11-03,1 percent chance,2020-06-21,299042.0,2000000.0,True,False,200,...,48 hours,33792,226000.0,0.0,135,False,0,"Medical, Illness & Healing",0.0,0.0
2,3,https://www.gofundme.com/f/100-computers-in-10...,2020-11-04,MELS needs #100ComputersIn100Days,2020-06-19,13845.0,30000.0,True,True,200,...,48 hours,13845,30000.0,8196.0,138,False,0,Others,7.0,4.0
3,4,https://www.gofundme.com/f/100-to-victims-of-t...,2020-11-03,100% to Victims of Taal Volcano in Philippines,2020-01-16,25.0,5000.0,True,True,200,...,48 hours,25,5000.0,0.0,292,False,0,Accidents & Emergencies,0.0,0.0
4,5,https://www.gofundme.com/f/100k-for-center-for...,2020-11-04,$100k for Center for Family Life,2020-11-02,13950.0,100000.0,True,False,200,...,48 hours,13950,100000.0,1100.0,2,False,0,Others,1.0,0.0


In [26]:
general.columns

Index(['chid', 'url', 'scrape_date', 'title', 'created_date', 'funds_raised',
       'funds_target', 'picture', 'org_picture', 'counts_status_code',
       'velocity_status_code', 'updates_status_code', 'comments_status_code',
       'donations_status_code', 'currency', 'category', 'story',
       'receiver_name', 'receiver_tax', 'total_photos', 'total_co_photos',
       'total_community_photos', 'total_updates', 'total_donations',
       'total_unique_donors', 'amount_raised_unattributed',
       'number_of_donations_unattributed', 'campaign_hearts',
       'social_share_total', 'num_recent_donations', 'recent_time_window',
       'funds_raised_dollar', 'funds_target_dollar',
       'amount_raised_unattributed_dollar', 'char_age', 'success',
       'dum_success', 'group_category', 'total_comments', 'update_counts'],
      dtype='object')

In [27]:
# 카테고리 내용 확인 
general.category.value_counts()

Medical, Illness & Healing     1117
Accidents & Emergencies        1104
Funerals & Memorials           1030
Community & Neighbors           193
Education & Learning             97
Other                            72
Volunteer & Service              58
Animals & Pets                   52
Missions, Faith & Church         48
Creative Arts, Music & Film      36
Sports, Teams & Clubs            32
Babies, Kids & Family            26
Business & Entrepreneurs         12
Environment                       9
Dreams, Hopes & Wishes            7
Celebrations & Events             5
Non-Profits & Charities           2
Competitions & Pageants           2
Travel & Adventure                1
Rent, Food & Monthly Bills        1
Name: category, dtype: int64

In [28]:
# 카테고리 4개로 나누기 
    # Medical, Illness & Healing
    # Accidents & Emergencies
    # Funerals & Memorials
    # Others

def changecategory(x): 
    if x == 'Medical, Illness & Healing': 
        return 'Medical, Illness & Healing'
    
    elif x == 'Accidents & Emergencies': 
        return 'Accidents & Emergencies'
    
    elif x == 'Funerals & Memorials': 
        return 'Funerals & Memorials'
    
    else: 
        return 'Others'

In [29]:
general.category= general.category.apply(changecategory)

In [30]:
# 새롭게 정의된 카테고리 범주 
general.category.value_counts()

Medical, Illness & Healing    1117
Accidents & Emergencies       1104
Funerals & Memorials          1030
Others                         653
Name: category, dtype: int64

# 카테고리 별 story 길이 확인 

In [45]:
general.story = general.story.astype('str')

In [56]:
# 스토리마다 글자갯수 구해서 story_lenth 칼럼 만들기 
general['story_length'] = general['story'].map(lambda calc: len(calc))
general['story_length'].to_frame()

,story_length
0,1506
1,5152
2,6270
3,402
4,1666
...,...
3899,1399
3900,1423
3901,3135
3902,617


In [60]:
general[['category', 'story','story_length']]

,category,story,story_length
0,Funerals & Memorials,This is the official Chris Beaty GoFundMe crea...,1506
1,"Medical, Illness & Healing",Shortly about Saha: | It is with a heavy heart...,5152
2,Others,Cost Breakdown | 100 Computers x approximately...,6270
3,Accidents & Emergencies,I was caught within 7 miles of the Taal Volcan...,402
4,Others,I have been on the board of the Center for Fam...,1666
...,...,...,...
3899,"Medical, Illness & Healing","In late June, Matt Hansen was diagnosed with a...",1399
3900,Funerals & Memorials,Scott Zurn was diagnosed with an aggressive fo...,1423
3901,"Medical, Illness & Healing","First, and foremost, let��셲 talk about Sean. S...",3135
3902,Funerals & Memorials,"On Monday August 3, 2020, Koorosh Yaghoubian (...",617


In [134]:
# 카테고리별 평균 스토리 길이 
general.groupby('category')['story_length'].mean().to_frame().reset_index().sort_values(by='story_length', ascending = False)

,category,story_length
2,"Medical, Illness & Healing",2654.550582
3,Others,2103.298622
0,Accidents & Emergencies,1760.616848
1,Funerals & Memorials,1285.424272


- Others 를 제외한 top3 카테고리에서 확실히 Funerals & Memorials의 평균 스토리 길이가 짧은 것으로 나타났다.  

# 카테고리 별 성공여부 확인 

In [119]:
general['gensuccess'] = general.funds_raised_dollar > general.funds_target_dollar
general['dum_gensuccess']= general['gensuccess'].astype(int)

In [138]:
category_success_sum = general.groupby('category')['dum_gensuccess'].sum().to_frame().reset_index().sort_values(by='dum_gensuccess', ascending = False)
category_success_sum

,category,dum_gensuccess
1,Funerals & Memorials,517
2,"Medical, Illness & Healing",413
0,Accidents & Emergencies,410
3,Others,216


- Funerals & Memorials 의 성공 갯수가 다른 카테고리에 비해 높게 나왔다. 

# 카테고리별 total_comments, update_counts 보기 

In [141]:
general[['category','total_comments','update_counts']]

,category,total_comments,update_counts
0,Funerals & Memorials,74.0,1.0
1,"Medical, Illness & Healing",0.0,0.0
2,Others,7.0,4.0
3,Accidents & Emergencies,0.0,0.0
4,Others,1.0,0.0
...,...,...,...
3899,"Medical, Illness & Healing",17.0,0.0
3900,Funerals & Memorials,27.0,0.0
3901,"Medical, Illness & Healing",41.0,3.0
3902,Funerals & Memorials,1.0,0.0


In [148]:
# 카테고리에 따른 코멘트와 업데이트 갯수의 평균을 확인 
general.groupby('category')[['total_comments', 'update_counts']].mean().reset_index().sort_values(by='total_comments', ascending = False)

,category,total_comments,update_counts
2,"Medical, Illness & Healing",58.256938,5.166517
0,Accidents & Emergencies,46.359601,3.645833
1,Funerals & Memorials,40.201942,1.771845
3,Others,18.770291,2.160796


- top 3 중 Funerals & Memorials 는 평균 코멘트 갯수와 평균 업데이트 횟수가 가장 낮은 편이었다. 